# Main

In [ ]:
# import the dataset from gsheets
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('dataset').sheet1

In [ ]:
# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
# print(rows)

import pandas as pd
df = pd.DataFrame.from_records(rows)

# creating columns name
df.columns = df.iloc[0]
df = df.iloc[1:]

# ChatGPT

Paraphrase using https://chat.openai.com/chat website

# GPT-3

In [ ]:
!pip install openai

In [ ]:
import openai
import time

openai.organization = ""
openai.api_key = ""

for i, reference in df["reference"].items():
  if df.loc[i, "gpt3"] != '':
    continue
  else:
    try:
      response = openai.Completion.create(
          engine="text-davinci-003", 
          prompt=f"paraphrase:{reference}",
          # prompt=f"Act as AI sentence paraphraser and just paraphrase:{reference}",
          max_tokens=512
          )
      df.loc[i, "gpt3"] = response["choices"][0]["text"].replace("\n", "")
      time.sleep(1)
    except:
      print(f'{i} - Error')
      time.sleep(5)
      continue
  # if i == 10000:
  #   break

In [ ]:
# update the gsheets
cell_list = worksheet.range("D2:D10002")

for i, val in enumerate(df["gpt3"]):
  # if cell_list[i].value != '':
  #   continue
  # if val == '':
  #   print(i)
  #   break
  cell_list[i].value = val
  # print(i)

worksheet.update_cells(cell_list)

# T5

In [ ]:
# !pip install git+https://github.com/PrithivirajDamodaran/Parrot.git
!pip install git+https://github.com/massyakur/Parrot_Paraphraser

In [ ]:
# import libraries
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# for reproducibility
def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

random_state(1234)

In [ ]:
# init the fine-tuned model (make sure you init ONLY once if you integrate this to your code)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5") # , use_gpu=False)

In [ ]:
# stop_n = 10000

for i, reference in df["reference"].iteritems():
  if df.loc[i, "t5_0"] != '':
    continue
  else:
    try:
      responses = parrot.augment(input_phrase=reference, 
                                use_gpu=True,
                                do_diverse=False,
                                diversity_ranker="levenshtein",
                                adequacy_threshold = 0.5,
                                fluency_threshold = 0.5
                                )
      for j, response in enumerate(responses):
        column_name = f"t5_{j}"
        df.loc[i, column_name] = response[0]
        # print(f"{i}-{column_name} = {response[0]}")
        if j == 2:
          break
    except:
      print(f"{i} - No paraphrases returned")
  # if i == stop_n:
  #   break